## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'],inplace=True)

In [21]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [22]:
# Look at APPLICATION_TYPE value counts for binning
application_df.groupby('APPLICATION_TYPE').size().sort_values(ascending=False)


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [23]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [24]:
# Look at CLASSIFICATION value counts for binning
application_df.groupby('CLASSIFICATION').size().sort_values(ascending=False)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [25]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_df=application_df.groupby('CLASSIFICATION').size().sort_values(ascending=False)
class_df=class_df.reset_index()
class_df = class_df.loc[class_df[class_df.index[0]]>1]
print(class_df)

   CLASSIFICATION      0
0           C1000  17326
1           C2000   6074
2           C1200   4837
3           C3000   1918
4           C2100   1883
5           C7000    777
6           C1700    287
7           C4000    194
8           C5000    116
9           C1270    114
10          C2700    104
11          C2800     95
12          C7100     75
13          C1300     58
14          C1280     50
15          C1230     36
16          C1400     34
17          C2300     32
18          C7200     32
19          C1240     30
20          C8000     20
21          C7120     18
22          C1500     16
23          C1800     15
24          C6000     15
25          C1250     14
26          C8200     11
27          C1278     10
28          C1238     10
29          C1235      9
30          C1237      9
31          C7210      7
32          C1720      6
33          C4100      6
34          C2400      6
35          C1600      5
36          C1257      5
37          C2710      3
38          C1260      3


In [26]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=['C7000',
'C1700',
'C4000',
'C5000',
'C1270',
'C2700',
'C2800',
'C7100',
'C1300',
'C1280',
'C1230',
'C1400',
'C2300',
'C7200',
'C1240',
'C8000',
'C7120',
'C1500',
'C1800',
'C6000',
'C1250',
'C8200',
'C1278',
'C1238',
'C1235',
'C1237',
'C7210',
'C1720',
'C4100',
'C2400',
'C1600',
'C1257',
'C2710',
'C1260',
'C1267',
'C1246',
'C1256',
'C3200',
'C1234',
'C0',
'C5200',
'C1236',
'C2190',
'C4200',
'C1248',
'C1370',
'C2600',
'C1580',
'C6100',
'C1820',
'C1900',
'C2570',
'C1570',
'C3700',
'C2500',
'C1283',
'C2380',
'C1732',
'C1728',
'C2170',
'C4120',
'C8210',
'C4500',
'C1245',
'C2561',
'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_application_type =      pd.get_dummies(application_df['APPLICATION_TYPE'])
dummy_affiliation =           pd.get_dummies(application_df['AFFILIATION'])
dummy_classification =        pd.get_dummies(application_df['CLASSIFICATION'])
dummy_use_case =              pd.get_dummies(application_df['USE_CASE'])
dummy_organization =          pd.get_dummies(application_df['ORGANIZATION'])
dummy_income_amt =            pd.get_dummies(application_df['INCOME_AMT'])
dummy_special_considerations = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'])


application_df = application_df.drop(columns='APPLICATION_TYPE')
application_df = application_df.drop(columns='AFFILIATION')
application_df = application_df.drop(columns='CLASSIFICATION')
application_df = application_df.drop(columns='USE_CASE')
application_df = application_df.drop(columns='ORGANIZATION')
application_df = application_df.drop(columns='INCOME_AMT')
application_df = application_df.drop(columns='SPECIAL_CONSIDERATIONS')

application_df_transformed=pd.concat([application_df,dummy_application_type,dummy_affiliation,dummy_classification,dummy_use_case,dummy_organization,dummy_income_amt,dummy_special_considerations], axis=1)
application_df_transformed

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M,N,Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


In [29]:
# Split our preprocessed data into our features and target arrays
target=application_df_transformed["IS_SUCCESSFUL"]
data = application_df_transformed.drop("IS_SUCCESSFUL", axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [46]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 16)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 993 (3.88 KB)

 Trainable params: 993 (3.88 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.7345 - loss: 0.5417
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - accuracy: 0.7396 - loss: 0.5377
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 638us/step - accuracy: 0.7424 - loss: 0.5339
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 683us/step - accuracy: 0.7431 - loss: 0.5345
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 655us/step - accuracy: 0.7400 - loss: 0.5340
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 661us/step - accuracy: 0.7389 - loss: 0.5369
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 625us/step - accuracy: 0.7424 - loss: 0.5322
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - accuracy: 0.7340 - loss: 0.5447
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 711us/step - accuracy: 0.7361 - loss: 0.5394
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - accuracy: 0.7391 - loss: 0.5355
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 670us/step - accuracy: 0.7349 - loss: 0.5404
Epoch 12/50
804/804 ━━━━━━━━━━

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 582us/step - accuracy: 0.7322 - loss: 0.5599
Loss: 0.5598908066749573, Accuracy: 0.7322449088096619


In [41]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')
nn.save_weights('my_model.weights.h5')